## import

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import pandas as pd

/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <BE0CCD9A-269A-30E2-A23C-DA45E89EBB1F> /Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/lib-dynload/../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/bin/../lib/libjpeg.8.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS@rpath/libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../

# load data

### define directory path

In [2]:
import os
import shutil

# 데이터셋 경로 설정하기
dataset_path = './usg1_dataset/train_images/'
dir = "./usg1_dataset/"
image_dir = dir+'train_images/'
train_csv_dir = dir+'train.csv'
train_copy_dir = './usg1_dataset/train_copy/'
test_dir = './usg1_dataset/test_images/'
split_label_dir = dir + 'train_split_label/'

### split train/valid/test

In [3]:
os.makedirs(split_label_dir, exist_ok=True)

train_dir = split_label_dir + 'train/'
valid_dir = split_label_dir + 'valid/'
test_dir = split_label_dir + 'test/'

os.makedirs(split_label_dir + '/train', exist_ok=True)
os.makedirs(split_label_dir + '/valid', exist_ok=True)
os.makedirs(split_label_dir + '/test', exist_ok=True)


shutil.rmtree(train_dir)
shutil.rmtree(valid_dir)
shutil.rmtree(test_dir)

os.makedirs(split_label_dir + '/train', exist_ok=True)
os.makedirs(split_label_dir + '/valid', exist_ok=True)
os.makedirs(split_label_dir + '/test', exist_ok=True)


# 레이블 이름 설정하기
train_labels = pd.read_csv(train_csv_dir)
labels = train_labels['label'].to_list() # 레이블 리스트
labels_str = []
for label in labels:
    labels_str.append(str(label))

# 하위 폴더 생성하기
for label in labels_str:
    os.makedirs(train_dir + label, exist_ok=True)
    os.makedirs(valid_dir + label, exist_ok=True)

### copy : train, test

In [4]:
import os
from natsort import natsorted

# 파일 이름 변경하기 ""

# 1) copy : train_images => train_copy (and test)

# train
#os.makedirs(train_copy_dir)
shutil.rmtree(train_copy_dir)
os.makedirs(train_copy_dir)

filenames = natsorted(os.listdir(image_dir))
for filename in filenames:
    if filename.endswith('.jpg'):  # 변경할 파일 확장자 지정

        name = os.path.join(image_dir, filename)
        shutil.copy(name, train_copy_dir)


# test
test_dir = './usg1_dataset/test_images/'
split_test_dir = split_label_dir + 'test/'

test_filenames = natsorted(os.listdir(test_dir))

for filename in test_filenames:
    if filename.endswith('.jpg'):  # 변경할 파일 확장자 지정

        name = os.path.join(test_dir, filename)
        shutil.copy(name, split_test_dir)

### rename : train

In [5]:
filenames = natsorted(os.listdir(train_copy_dir))

# 2) rename
for idx, filename in enumerate(filenames):
        label = labels_str[idx]

        name = os.path.join(train_copy_dir, filename)
        labeled_name = os.path.join(train_copy_dir, os.path.splitext(filename)[0]+ '_' +label+'.jpg')
        os.rename(name, labeled_name)

### split : train, valid

In [6]:
# 2) split 
from sklearn.model_selection import train_test_split

filenames = natsorted(os.listdir(train_copy_dir))

train_filenames, valid_filenames = train_test_split(filenames, test_size=0.1, random_state=42)

train_filenames = natsorted(train_filenames)
valid_filenames = natsorted(valid_filenames)

print('train\n',train_filenames[:5])
print('valid\n',valid_filenames[:5])

train
 ['000000_1.jpg', '000001_8.jpg', '000002_0.jpg', '000003_1.jpg', '000004_8.jpg']
valid
 ['000008_9.jpg', '000014_1.jpg', '000019_0.jpg', '000031_0.jpg', '000033_6.jpg']


In [7]:
# 이미지 파일 이동하기

# train
for file in train_filenames:
    if file.endswith('.jpg'):
        filename = os.path.splitext(file)[0]
        label = filename.split('_')[1]
        shutil.copy(train_copy_dir+ '/' +file, train_dir + label + '/' + file)

# valid
for file in valid_filenames:
    if file.endswith('.jpg'):
        filename = os.path.splitext(file)[0]
        label = filename.split('_')[1]
        shutil.copy(train_copy_dir+ '/' +file, valid_dir + label + '/' + file)

# Model

### transforms

In [8]:
import tqdm

import tqdm

from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize, RandomVerticalFlip, RandomRotation, RandomResizedCrop, RandomGrayscale, RandomSolarize
from torch.optim.adam import Adam

transforms = Compose([
   Resize(224),
   RandomCrop((224, 224), padding=4),
   #RandomGrayscale(p=0.75),
   #RandomSolarize,
   #RandomVerticalFlip(), # 상하반전
   # RandomRotation(180), # 회전
   RandomHorizontalFlip(p=0.5), # 좌우반전
   ToTensor(),
   Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

### load DataLoader

In [9]:
# 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir,
                                      transform=transforms)

valid_dataset = datasets.ImageFolder(root=valid_dir,
                                      transform=transforms)


### Define model

In [10]:
import torch
import torch.nn as nn

from torchvision.models.resnet import resnet18, resnet34, resnet50, resnet152



device = torch.device('mps')

resnet = resnet18
model = resnet(pretrained=True) # ❶ resnet18 모델 객체 생성

model.to(device)

/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
# parameter
lr = 1e-4
epochs = 50
batch_size = 16

# model describe
model_name = f"0310_model_{str(resnet)[10:18]}_(lr={lr}_epochs={epochs}_batch_size={batch_size})"
print(str(resnet)[10:18])
print(model_name)

resnet18
0310_model_resnet18_(lr=0.0001_epochs=50_batch_size=16)


### fit

In [12]:
from torch.optim.adam import Adam

optim = Adam(model.parameters(), lr=lr)


# 데이터로더 생성하기
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)


for epoch in range(epochs):
   iterator = tqdm.tqdm(train_loader)
   for data, label in iterator:
       # 최적화를 위해 기울기를 초기화
       optim.zero_grad()

       # 모델의 예측값
       preds = model(data.to(device))

       # 손실 계산 및 역전파
       loss = nn.CrossEntropyLoss()(preds, label.to(device))
       loss.backward()
       optim.step()

       iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")


# save model
torch.save(model.state_dict(), model_name+".pth")

epoch:50 loss:0.0003830294299405068: 100%|██████████| 716/716 [00:47<00:00, 15.08it/s] 


### Accuracy

In [14]:
print(model_name)
model.load_state_dict(torch.load(model_name+".pth", map_location=device))

num_corr = 0

with torch.no_grad():
   for data, label in valid_loader:

       output = model(data.to(device))
       preds = output.data.max(1)[1]
       corr = preds.eq(label.to(device).data).sum().item()
       num_corr += corr

   print(f"Accuracy:{num_corr/len(valid_dataset)}")

0310_model_resnet18_(lr=0.0001_epochs=50_batch_size=16)
Accuracy:0.9457547169811321


resnet18  
- (batch=16/epochs=10) Accuracy:0.9363207547169812  
- (batch=16/epochs=30)  

resnet34
- (batch=16/epochs=10) Accuracy:0.9504716981132075  
- (batch=16/epochs=30) Accuracy:0.9559748427672956 👍


resnet50  
- (batch=16/epochs=10) Accuracy:0.9323899371069182
- (epochs=30_batch_size=32) Accuracy:0.9520440251572327

 
resnet152   
- 

### Submit

In [15]:
from torch.utils.data import Dataset
from PIL import Image

class UnlabeledDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform

        self.filelist = natsorted(os.listdir(data_path))

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_path, self.filelist[idx])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

In [17]:
test_dataset = UnlabeledDataset(data_path=split_test_dir, transform=transforms)

batch_size = 32

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)



# predict
model.load_state_dict(torch.load(model_name+".pth", map_location=device))

sub = []

with torch.no_grad():
   for data in test_loader:

       output = model(data.to(device))
       preds = output.data.max(1)[1]

       preds = preds.cpu()
       preds = preds.numpy()

       for i in range(len(preds)):
        sub.append(preds[i])

submit_df = pd.DataFrame(data=natsorted(os.listdir(split_test_dir)), columns=['image_name'])
submit_df['label'] = sub

In [18]:
submit_df.to_csv(f'{model_name}.csv')

In [19]:
model_name

'0310_model_resnet18_(lr=0.0001_epochs=50_batch_size=16)'